In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-25"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
119,2024-08-25,Eua Wnba,17:00,Dallas Wings F,Los Angeles Sparks F,1.62,2.31,166.5,1.87,1.92,-3.5,1.92,2.75,OdG1CHdt,0.617284,0.432900,0.534759,0.520833,0.050184,0.0,0.0,NaN,NaN,174.85,62.611497,0.358087,179.524,16.775364,0.093444,0.0,0.0,266.64,205.32,1.0,1.0,0.0,0.0,0.248297,0.018657,0.251348,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
120,2024-08-25,Filipinas Copa Dos Governadores,06:00,Phoenix Fuelmasters,NLEX Road Warriors,2.00,1.76,209.5,1.87,1.79,1.5,1.86,1.60,pfsabyHb,0.500000,0.568182,0.534759,0.558659,0.068182,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.090269,0.030912,0.106270,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
121,2024-08-25,Porto Rico Bsn,21:10,Criollos de Caguas,Osos de Manati,1.61,2.12,183.5,1.82,1.84,-3.5,1.93,2.37,Y9d1dCvf,0.621118,0.471698,0.549451,0.543478,0.092816,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,170.98,0.0,0.0,0.0,0.0,0.193364,0.007728,0.144710,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
122,2024-08-25,Vietnã Vba,09:30,Can Tho Catfish,Da Nang Dragons,1.77,1.91,155.5,1.83,1.83,-2.5,2.01,2.40,roDZwUpm,0.564972,0.523560,0.546448,0.546448,0.088532,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,148.92,0.00,0.0,0.0,0.0,0.0,0.053802,0.000000,0.125067,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
123,2024-08-25,República Dominicana Lnb,20:00,Metros de Santiago,Reales de la Vega,1.97,1.71,172.5,1.80,1.86,1.5,1.85,1.61,0IJGoWnA,0.507614,0.584795,0.555556,0.537634,0.092410,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,129.36,164.82,0.0,0.0,0.0,0.0,0.099917,0.023184,0.098096,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,17:00,Eua Wnba,Dallas Wings F,Los Angeles Sparks F,166.5,1.87,1.0,Over
1,21:10,Porto Rico Bsn,Criollos de Caguas,Osos de Manati,183.5,1.82,1.0,Over
2,09:30,Vietnã Vba,Can Tho Catfish,Da Nang Dragons,155.5,1.83,1.0,Over
